[View in Colaboratory](https://colab.research.google.com/github/iamrajee/Machine-Learning/blob/master/trans_learning_keras.ipynb)

In [1]:
!ls

datalab  drive


In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}





# !mkdir -p drive
# !google-drive-ocamlfuse drive


# !ls

In [0]:
import pandas as pd
import numpy as np
y_train_df = pd.read_csv("drive/ml_data/transfer_learning/train_date_data/datefinal.csv")

In [4]:
y_train_df

,Unnamed: 0,0
0,0,1930
1,1,1935
2,2,1930
3,3,1960
4,4,1945
5,5,1950
6,6,1926
7,7,1976
8,8,1930
9,9,1960


In [5]:
y_train_df.columns

Index(['Unnamed: 0', '0'], dtype='object')

In [6]:
y_all = y_train_df['0']
print(max(y_all))
print(min(y_all))

1980
1902


In [0]:
for i,ele in enumerate(y_all):
  y_all[i] = (ele-1900)//5

In [8]:
print(y_all[6])

print(max(y_all))
print(min(y_all))

5
16
0


In [9]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(y_all)
print(transfomed_label)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
print(transfomed_label[799])

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]


In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [14]:
import keras


from keras import applications
from keras import optimizers
from keras import backend as k

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard,EarlyStopping

Using TensorFlow backend.


In [0]:
img_width, img_height = 256, 256

# nb_train_samples = samples = 
# nb_validation_samples = 

batch_size = 16
epochs = 50

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))



In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [0]:
#freeze the few(1st five) layers for undesirable change

for layer in model.layers[:5]:
  layer.trainable = False


In [0]:
#adding custom layers
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024,activation="relu")(x)
predictions = Dense(17,activation="softmax")(x)

In [38]:
#create final model
model_final = Model(input=model.input, output = predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [0]:
#compiling the model
model_final.compile(loss="categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9),metrics=["accuracy"])

In [40]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [0]:
train_data_dir = "drive/ml_data/transfer_learning/train_image_data"
validation_data_dir = "drive/ml_data/transfer_learning/test_image_data"

In [0]:
train_datagen = ImageDataGenerator(
                   rescale = 1./255,
                   horizontal_flip = True,
                   fill_mode = "nearest",
                   zoom_range = 0.3,
                   width_shift_range = 0.3,
                   height_shift_range=0.3,
                   rotation_range=30)

test_datagen = ImageDataGenerator(
                  rescale = 1./255,
                  horizontal_flip = True,
                  fill_mode = "nearest",
                  zoom_range = 0.3,
                  width_shift_range = 0.3,
                  height_shift_range=0.3,
                  rotation_range=30)

In [43]:
train_generator = train_datagen.flow_from_directory(
                                      train_data_dir,
                                      target_size = (img_height, img_width),
                                      batch_size = batch_size, 
                                      class_mode = "categorical")


Found 0 images belonging to 0 classes.


In [44]:
validation_generator = test_datagen.flow_from_directory(
                                      validation_data_dir,
                                      target_size = (img_height, img_width),
                                      class_mode = "categorical")

Found 0 images belonging to 0 classes.


In [0]:
nTrain = 800



datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
 
train_features = np.zeros(shape=(nTrain, 8, 8, 512))
train_labels = np.zeros(shape=(nTrain,3))

In [55]:
# i = 0
# for inputs_batch, labels_batch in train_generator:
#     features_batch = model.predict(inputs_batch)
#     train_features[i * batch_size : (i + 1) * batch_size] = features_batch
#     train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
#     i += 1
#     if i * batch_size >= nImages:
#         break
         
# train_features = np.reshape(train_features, (nTrain, 8 * 8 * 512))

ZeroDivisionError: ignored

In [0]:
model_final.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])


In [0]:
nb_train_samples = 800
nb_validation_samples = 110

# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])


In [61]:
# history = model_final.fit(train_features,
#                     train_labels,
#                     epochs=20,
#                     batch_size=batch_size,
#                     validation_data=(validation_features,validation_labels))


NameError: ignored